<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/03_Morphopmetrics_on_binary_classification_true_positives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Perform Morphometrics on True Positive Outputs



*   02 morphometric script on true and false positives, this one only on true positives



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
Binary_Classification_notebooks  Detectron2_notebooks  Morphometrics_notebooks	src
detectron2			 Mask-RCNN	       output


## Write CSV for True Positive from Binary Classification

In [2]:
import pandas as pd

# Define the file paths
input_file_path = "/content/drive/MyDrive/data/species_53/Binary_Classification/model_save/epoch_100_dataframe.csv"
output_file_path = "/content/drive/MyDrive/data/species_53/Binary_Classification/model_save/nannofossil_prediction_true_positives.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file_path)

# Filter the DataFrame to include only rows where groundtruth_labels are 1 and Predictions are greater than 0.5
filtered_df = df[(df['groundtruth_labels'] == 1) & (df['Predictions'] > 0.5)]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered CSV saved to {output_file_path}")

Filtered CSV saved to /content/drive/MyDrive/data/species_53/Binary_Classification/model_save/nannofossil_prediction_true_positives.csv


## Get corresponding masks from Mask R-CNN

In [3]:
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import yaml

In [4]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 19.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.

In [5]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [6]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [7]:
# Import Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog, detection_utils as utils, build_detection_train_loader
from detectron2.structures import BoxMode

In [8]:
# Load the list of correctly predicted cropped image names from the CSV
pred_csv_path = "/content/drive/MyDrive/data/species_53/Binary_Classification/model_save/nannofossil_prediction_true_positives.csv"
pred_df = pd.read_csv(pred_csv_path)
correct_predictions = set(pred_df['image_name'].tolist())

In [9]:
# Define the file paths for the datasets
train_json = "/content/drive/MyDrive/data/species_53/data/train/coco_train.json"
train_images_path = "/content/drive/MyDrive/data/species_53/data/train"
val_json = "/content/drive/MyDrive/data/species_53/data/val/coco_val.json"
val_images_path = "/content/drive/MyDrive/data/species_53/data/val"

In [10]:
# Register the datasets
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, train_json, train_images_path)
register_coco_instances("my_dataset_val", {}, val_json, val_images_path)

In [11]:
# Extract the metadata and dataset dictionaries
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

[08/01 11:14:52 d2.data.datasets.coco]: Loading /content/drive/MyDrive/data/species_53/data/train/coco_train.json takes 2.16 seconds.
[08/01 11:14:52 d2.data.datasets.coco]: Loaded 289 images in COCO format from /content/drive/MyDrive/data/species_53/data/train/coco_train.json
[08/01 11:14:54 d2.data.datasets.coco]: Loading /content/drive/MyDrive/data/species_53/data/val/coco_val.json takes 1.61 seconds.
[08/01 11:14:54 d2.data.datasets.coco]: Loaded 95 images in COCO format from /content/drive/MyDrive/data/species_53/data/val/coco_val.json


In [12]:
# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"  # Path to your saved config file
cfg = get_cfg()

# Set default values for keys that might be missing in the config file
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Merge the configuration file with the default (debugged this way)
cfg.merge_from_file(config_yaml_path)
# Initialize the predictor with the loaded configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")

In [13]:
predictor = DefaultPredictor(cfg)

[08/01 11:15:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


In [14]:
# Create the output directory for segmented masks
output_dir = "/content/drive/MyDrive/data/species_53/Binary_Classification/true_positive_masks_for_morphometrics"
os.makedirs(output_dir, exist_ok=True)

In [15]:
# Function to process and save segmented masks
def process_and_save_segmented_masks(dataset_dicts):
    for d in dataset_dicts:
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        masks = outputs["instances"].pred_masks.to("cpu").numpy()
        num_instances = masks.shape[0]

        for i in range(num_instances):
            base_name = os.path.basename(d["file_name"])
            file_name, file_ext = os.path.splitext(base_name)
            cropped_image_name = f"{file_name}_{i+1}{file_ext}"

            if cropped_image_name in correct_predictions:
                item_mask = masks[i]
                segmentation = np.where(item_mask == True)
                x_min = int(np.min(segmentation[1]))
                x_max = int(np.max(segmentation[1]))
                y_min = int(np.min(segmentation[0]))
                y_max = int(np.max(segmentation[0]))

                mask_cropped = item_mask[y_min:y_max, x_min:x_max]
                output_file_name = f"{file_name}_{i+1}_mask{file_ext}"
                output_path = os.path.join(output_dir, output_file_name)
                mask_image = Image.fromarray((mask_cropped * 255).astype(np.uint8), mode='L')
                mask_image.save(output_path)

# Process and save segmented masks for train and val datasets
process_and_save_segmented_masks(train_dataset_dicts)
process_and_save_segmented_masks(val_dataset_dicts)

print("Segmented masks saved successfully for correctly predicted cropped images.")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Segmented masks saved successfully for correctly predicted cropped images.


## Run Morphometrics on masks

In [16]:
from skimage.measure import label, regionprops
from tqdm import tqdm
import csv

In [17]:
# Function to calculate morphometrics for all masks
def calculate_morphometrics(image_path):
    # Verify that the image exists before loading
    if not os.path.exists(image_path):
        print(f"Warning: Image {os.path.basename(image_path)} does not exist. Skipping.")
        return []

    new_im = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
    if new_im is None:
        print(f"Warning: Image {os.path.basename(image_path)} could not be loaded. Skipping.")
        return []

    # Apply binary thresholding to generate the mask
    threshold_value = 50  # Set the threshold value
    _, binary_image = cv2.threshold(new_im, threshold_value, 255, cv2.THRESH_BINARY)  # Convert to binary image
    labeled_mask = label(binary_image)  # Label connected components

    # Check if the mask is empty
    if np.count_nonzero(labeled_mask) == 0:
        print(f"Warning: Image {os.path.basename(image_path)} has an empty mask. Skipping.")
        return []

    props = regionprops(labeled_mask)
    if not props:
        print(f"Warning: No properties found for image {os.path.basename(image_path)}. Skipping.")
        return []

    # Define scale factor for converting pixels to micrometers
    scale = 0.0735
    results = []

    for i, prop in enumerate(props):
        object_number = i + 1
        major_axis_length = prop.major_axis_length * scale
        minor_axis_length = prop.minor_axis_length * scale

        # Ensure lengths are non-zero to avoid division by zero
        if major_axis_length == 0 or minor_axis_length == 0:
            print(f"Warning: Image {os.path.basename(image_path)}, Object {object_number} has zero length axis. Skipping.")
            continue

        area = prop.area * (scale ** 2)
        circularity = np.sqrt((minor_axis_length * major_axis_length) / (major_axis_length ** 2))
        ellipticity = major_axis_length / minor_axis_length

        results.append([os.path.basename(image_path), "t.orionatus", object_number, major_axis_length, minor_axis_length, circularity, ellipticity, area])

    return results

In [18]:
# Define the directories
masked_segmentations_dir = "/content/drive/MyDrive/data/species_53/Binary_Classification/true_positive_masks_for_morphometrics"
output_csv_path = "/content/drive/MyDrive/data/species_53/Binary_Classification/morphometrics_output/morphometrics_true_positives.csv"

# Check if output directory exists
output_dir = os.path.dirname(output_csv_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [19]:
# Get the list of mask images to process
mask_images = [f for f in os.listdir(masked_segmentations_dir) if f.endswith('.jpeg') or f.endswith('.png')]

print(f"Found {len(mask_images)} mask images to process.")

print("Starting CSV writing process...")
try:
    with open(output_csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["file_name", "class_name", "object_number", "major_axis", "minor_axis", "circularity", "ellipticity", "area"])

        for image_name in tqdm(mask_images, desc="Calculating morphometrics"):
            image_path = os.path.join(masked_segmentations_dir, image_name)
            morphometrics = calculate_morphometrics(image_path)
            if morphometrics:
                csvwriter.writerows(morphometrics)
            else:
                print(f"No valid data to write for image {image_name}")

except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")

print("Morphometrics processing complete.")

Found 55 mask images to process.
Starting CSV writing process...


Calculating morphometrics: 100%|██████████| 55/55 [00:00<00:00, 157.13it/s]

Morphometrics processing complete.
